In [1]:
from __future__ import division
import os, sys
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches

import pandas as pd
from svgpathtools import parse_path, wsvg, svg2paths

### setup

In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('../../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','python') not in sys.path:
    sys.path.insert(1,os.path.join(proj_dir,'analysis','python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

import svg_rendering_helpers as srh
reload(srh)

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['graphical_conventions']

# which iteration name should we use?
## "good" iterations that we currently combine: ['run3_size4_waiting', 'run4_generalization']
iterationNames = ['run3_size4_waiting', 'run4_generalization']

### svg rendering

In [ ]:
# list of all gameids across iterationNames
gameids = coll.find({'iterationName': {'$in': iterationNames}}).distinct('gameid')

In [ ]:
# sample game
this_game = gameids[10]

In [ ]:
# sample stroke recs from a trial
this_trial = 2
stroke_recs = coll.find({ '$and': [{'gameid': this_game}, 
                                   {'eventType': 'stroke'}, 
                                   {'trialNum':this_trial}]}).sort('time')

In [ ]:
# get list of svgs
svg_list = srh.make_svg_list(stroke_recs)

In [ ]:
# grab sample stroke
this_stroke = svg_list[0]

In [ ]:
parsed1 = parse_path(svg_list[0])
parsed2 = parse_path(svg_list[1])

In [ ]:
## list of paths to render out in same svg
parsed = [parsed1,parsed2]

#### wsvg arguments
```
wsvg(paths=None, colors=None, filename=join, stroke_widths=None, nodes=None, node_colors=None, node_radii=None, openinbrowser=False, timestamp=False, margin_size=0.1, mindim=600, dimensions=None, viewbox=None, text=None, text_path=None, font_size=None, attributes=None, svg_attributes=None)
```

### what we need (tests on single sketches)

- render_svg
- svg_to_png
- svg2paths (already defined in svgpathtools)

In [ ]:
reload(srh)
## render out svg
srh.render_svg(parsed,base_dir=sketch_dir)

In [ ]:
## get svg path list
svg_paths = srh.generate_svg_path_list(os.path.join(sketch_dir,'svg'))

In [ ]:
## convert to png
srh.svg_to_png(svg_paths,base_dir=sketch_dir)

### scratch space for scaling up to render out whole dataset 